<a href="https://colab.research.google.com/github/RichardMackMarvel00/SPICE-2024/blob/main/File_2019_2023WVHTproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Run Packages


In [ ]:
!pip install statsmodels
!pip install pmdarima
!pip install seaborn
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn import linear_model

from pandas import to_datetime
import datetime
from datetime import datetime
import pandas as pd
import pandas as pd
from io import StringIO
#import pmdarima as pm
import itertools
import warnings
import matplotlib.pyplot as plt
from matplotlib import pyplot

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_predict

warnings.filterwarnings('ignore')
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Data Sets


In [ ]:
df23 = pd.read_csv('https://www.ndbc.noaa.gov/view_text_file.php?filename=51000h2023.txt.gz&dir=data/historical/stdmet/', delimiter= '\s+', index_col=False) # reading in the data
df22 = pd.read_csv('https://www.ndbc.noaa.gov/view_text_file.php?filename=51000h2022.txt.gz&dir=data/historical/stdmet/', delimiter= '\s+', index_col=False) # reading in the data
df21 = pd.read_csv('https://www.ndbc.noaa.gov/view_text_file.php?filename=51000h2021.txt.gz&dir=data/historical/stdmet/', delimiter= '\s+', index_col=False) # reading in the data
df20 = pd.read_csv('https://www.ndbc.noaa.gov/view_text_file.php?filename=51000h2020.txt.gz&dir=data/historical/stdmet/', delimiter= '\s+', index_col=False) # reading in the data
df19 = pd.read_csv('https://www.ndbc.noaa.gov/view_text_file.php?filename=51000h2019.txt.gz&dir=data/historical/stdmet/', delimiter= '\s+', index_col=False) # reading in the data

#only have 40 mins intervals for df23, df22, df21, & df20

In [ ]:
df23

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2023,01,01,00,00,123,4.2,6.3,99.00,99.00,99.00,999,1017.1,24.0,24.7,19.4,99.0,99.00
2,2023,01,01,00,10,122,4.1,6.4,2.67,13.79,8.96,297,1017.1,24.0,24.7,19.4,99.0,99.00
3,2023,01,01,00,20,120,4.0,6.3,99.00,99.00,99.00,999,1017.1,23.9,24.7,19.5,99.0,99.00
4,2023,01,01,00,30,110,3.7,5.5,99.00,99.00,99.00,999,1017.1,23.9,24.7,19.8,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52518,2023,12,31,23,10,177,4.8,6.2,2.56,17.39,9.28,329,1019.1,22.2,24.0,18.5,99.0,99.0
52519,2023,12,31,23,20,174,4.6,6.1,99.0,99.0,99.0,999,1018.9,22.6,24.0,17.9,99.0,99.0
52520,2023,12,31,23,30,168,5.3,6.9,99.0,99.0,99.0,999,1018.8,22.7,24.0,18.1,99.0,99.0
52521,2023,12,31,23,40,168,4.6,6.5,2.87,17.39,10.02,324,1018.8,22.8,24.0,17.7,99.0,99.0


In [ ]:
df23_clean = df23[df23['mm'].astype(str).str.endswith(('40'))]
df23_clean


,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
5,2023,01,01,00,40,104,3.8,5.8,2.49,13.79,8.80,285,1017.0,23.9,24.7,19.9,99.0,99.00
11,2023,01,01,01,40,112,4.6,6.9,2.61,13.79,8.96,301,1017.0,23.7,24.7,19.6,99.0,99.00
17,2023,01,01,02,40,111,3.4,5.9,2.59,12.90,8.96,308,1017.2,23.7,24.7,19.6,99.0,99.00
23,2023,01,01,03,40,114,4.3,7.0,2.69,12.90,9.26,312,1017.6,23.8,24.6,19.7,99.0,99.00
29,2023,01,01,04,40,126,3.8,6.1,2.55,13.79,9.21,305,1018.5,24.0,24.6,19.4,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52497,2023,12,31,19,40,134,5.0,7.1,2.76,17.39,9.47,327,1021.4,22.9,24.1,19.0,99.0,99.0
52503,2023,12,31,20,40,112,3.6,5.2,2.65,17.39,9.15,335,1021.6,22.8,24.0,19.2,99.0,99.0
52509,2023,12,31,21,40,150,5.1,6.8,2.5,17.39,8.73,332,1021.1,22.5,999.0,17.9,99.0,99.0
52515,2023,12,31,22,40,165,4.4,6.0,2.4,17.39,8.63,326,1019.8,22.6,24.0,18.1,99.0,99.0


In [ ]:
df22_clean = df22[df22['mm'].astype(str).str.endswith(('40'))]
df22_clean

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
5,2022,01,01,00,40,116,10.1,13.3,2.94,8.33,6.33,73,1011.9,22.2,23.4,18.8,99.0,99.00
11,2022,01,01,01,40,110,10.5,13.9,3.02,8.33,6.61,84,1012.2,22.0,23.3,18.5,99.0,99.00
17,2022,01,01,02,40,106,11.6,15.3,3.03,7.69,6.51,85,1012.3,22.3,23.3,18.5,99.0,99.00
23,2022,01,01,03,40,104,11.3,14.4,3.12,9.09,6.43,73,1012.7,22.7,23.3,18.2,99.0,99.00
29,2022,01,01,04,40,123,10.5,13.1,3.20,8.33,6.54,94,1013.2,22.5,23.3,18.7,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52455,2022,12,31,19,40,123,5.5,8.0,2.72,13.79,8.81,290,1019.9,23.8,24.5,20.1,99.0,99.0
52461,2022,12,31,20,40,119,4.6,7.1,2.46,13.79,8.65,284,1019.9,23.8,24.5,19.3,99.0,99.0
52467,2022,12,31,21,40,132,3.7,5.7,2.54,13.79,8.55,292,1019.3,23.8,24.6,19.2,99.0,99.0
52473,2022,12,31,22,40,112,3.6,5.7,2.5,13.79,8.51,289,1018.3,24.0,24.6,18.6,99.0,99.0


In [ ]:
df21_clean = df21[df21['mm'].astype(str).str.endswith(('40'))]
df21_clean

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
5,2021,01,01,00,40,67,6.7,9.6,3.06,17.39,8.29,328,1019.4,21.8,24.7,18.7,99.0,99.00
11,2021,01,01,01,40,71,7.4,9.5,2.92,19.05,8.35,298,1019.7,23.1,24.7,18.3,99.0,99.00
17,2021,01,01,02,40,63,7.7,9.6,3.07,17.39,8.31,336,1020.2,23.1,24.7,18.5,99.0,99.00
23,2021,01,01,03,40,48,7.6,10.0,3.30,16.00,8.85,320,1021.3,23.2,24.7,18.8,99.0,99.00
29,2021,01,01,04,40,57,8.1,10.1,3.38,17.39,9.08,321,1022.0,22.8,24.7,19.3,99.0,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51497,2021,12,31,19,40,99,10.3,13.8,3.02,9.09,6.56,62,1015.0,22.7,23.4,18.1,99.0,99.0
51503,2021,12,31,20,40,100,11.1,14.2,2.96,7.14,6.45,86,1014.9,22.8,23.4,17.4,99.0,99.0
51509,2021,12,31,21,40,104,11.1,13.9,2.92,10.0,6.41,46,1014.1,22.8,23.4,17.3,99.0,99.0
51515,2021,12,31,22,40,109,11.3,14.1,2.89,8.33,6.31,78,1013.1,22.8,23.4,17.0,99.0,99.0


In [ ]:
df20_clean = df20[df20['mm'].astype(str).str.endswith(('40'))]
df20_clean.head()

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
2650,2020,04,21,00,40,160,5.7,7.2,1.88,12.90,7.60,266,1017.0,23.4,24.1,18.3,99.0,99.00
2656,2020,04,21,01,40,170,4.3,6.0,1.75,12.90,7.60,287,1016.9,23.4,24.1,18.2,99.0,99.00
2662,2020,04,21,02,40,172,5.0,6.2,1.83,12.90,8.18,282,1016.9,23.6,24.1,18.1,99.0,99.00
2668,2020,04,21,03,40,165,5.0,6.3,1.66,12.90,7.39,274,1016.9,23.6,24.1,17.7,99.0,99.00
2674,2020,04,21,04,40,162,5.2,6.9,1.68,12.12,7.53,276,1017.1,23.4,24.0,18.0,99.0,99.00


In [228]:
df19_clean = df19[df20['mm'].astype(str).str.endswith((''))]
df19_clean.head()

,#YY,MM,DD,hh,mm,WDIR,WSPD,GST,WVHT,DPD,APD,MWD,PRES,ATMP,WTMP,DEWP,VIS,TIDE
0,#yr,mo,dy,hr,mn,degT,m/s,m/s,m,sec,sec,degT,hPa,degC,degC,degC,mi,ft
1,2018,12,31,23,50,90,8.4,9.8,2.87,10.00,7.11,81,1017.3,23.4,24.3,999.0,99.0,99.00
2,2019,01,01,00,50,78,7.0,8.5,2.54,10.00,6.80,90,1017.5,23.1,24.3,999.0,99.0,99.00
3,2019,01,01,01,50,92,8.7,10.1,2.61,10.00,6.70,90,1017.7,23.7,24.3,999.0,99.0,99.00
4,2019,01,01,02,50,90,8.4,10.9,2.78,10.00,7.00,61,1018.1,23.8,24.3,999.0,99.0,99.00


In [232]:
df23_clean.head()

,WVHT
Date,
2023-01-01 01:40:00,2.61
2023-01-01 02:40:00,2.59
2023-01-01 03:40:00,2.69
2023-01-01 04:40:00,2.55
2023-01-01 05:40:00,2.28


In [237]:
#df23_clean = df23_clean[['Date', 'WVHT']] # selecting columns
df23_clean.head() # looking at first five rows of arima_data

,WVHT
Date,
2023-01-01 01:40:00,2.61
2023-01-01 02:40:00,2.59
2023-01-01 03:40:00,2.69
2023-01-01 04:40:00,2.55
2023-01-01 05:40:00,2.28


In [236]:
#df22_clean = df22_clean[['Date', 'WVHT']] # selecting columns
df22_clean.head() # looking at first five rows of arima_data

,WVHT
Date,
2022-01-01 01:40:00,3.02
2022-01-01 02:40:00,3.03
2022-01-01 03:40:00,3.12
2022-01-01 04:40:00,3.20
2022-01-01 05:40:00,3.23


In [235]:
#df21_clean = df21_clean[['Date', 'WVHT']] # selecting columns
df21_clean.head() # looking at first five rows of arima_data

,WVHT
Date,
2021-01-01 01:40:00,2.92
2021-01-01 02:40:00,3.07
2021-01-01 03:40:00,3.30
2021-01-01 04:40:00,3.38
2021-01-01 05:40:00,3.50


In [234]:
#df20_clean_dates = df20[df20['mm'].astype(str).str.endswith(('40'))] # create a new dataframe with the filtered data
#df20_clean = df20_clean[['Date', 'WVHT']] # selecting columns from the original df20_clean
df20_clean.head() # looking at first five rows of arima_data

,WVHT
Date,
2020-04-21 01:40:00,1.75
2020-04-21 02:40:00,1.83
2020-04-21 03:40:00,1.66
2020-04-21 04:40:00,1.68
2020-04-21 05:40:00,1.71


In [259]:
df19_clean = df19_clean.astype({'#YY' : str, 'MM' : str, 'DD' : str, 'hh' : str, 'mm' : str}) # converting these columns to a string data type
df19_clean['Date'] = df19_clean['#YY'] + '-' + df19_clean['MM'] + '-' + df19_clean['DD'] + ' ' + df19_clean['hh'] + ':' + df19_clean['mm'] # combining all the datetime info into a Date column
df19_clean = df19_clean.drop(columns=['#YY','MM','DD','hh','mm']) # deleting the old date and time columns
df19_clean

KeyError: "Only a column name can be used for the key in a dtype mappings argument. '#YY' not found in columns."

In [261]:
df19_clean = df19_clean[['Date', 'WVHT']] # selecting columns from the original df20_clean
df19_clean.head() # looking at first five rows of arima_data

,Date,WVHT
2,2019-01-01 00:50,2.54
3,2019-01-01 01:50,2.61
4,2019-01-01 02:50,2.78
5,2019-01-01 03:50,2.58
6,2019-01-01 04:50,2.65


#Combined Clean

In [258]:
print(df23_clean.head())
print(df22_clean.head())
print(df21_clean.head())
print(df20_clean.head())
print(df19_clean.head())

                     WVHT
Date                     
2023-01-01 01:40:00  2.61
2023-01-01 02:40:00  2.59
2023-01-01 03:40:00  2.69
2023-01-01 04:40:00  2.55
2023-01-01 05:40:00  2.28
                     WVHT
Date                     
2022-01-01 01:40:00  3.02
2022-01-01 02:40:00  3.03
2022-01-01 03:40:00  3.12
2022-01-01 04:40:00  3.20
2022-01-01 05:40:00  3.23
                     WVHT
Date                     
2021-01-01 01:40:00  2.92
2021-01-01 02:40:00  3.07
2021-01-01 03:40:00  3.30
2021-01-01 04:40:00  3.38
2021-01-01 05:40:00  3.50
                     WVHT
Date                     
2020-04-21 01:40:00  1.75
2020-04-21 02:40:00  1.83
2020-04-21 03:40:00  1.66
2020-04-21 04:40:00  1.68
2020-04-21 05:40:00  1.71
               Date  WVHT
2  2019-01-01 00:50  2.54
3  2019-01-01 01:50  2.61
4  2019-01-01 02:50  2.78
5  2019-01-01 03:50  2.58
6  2019-01-01 04:50  2.65


#Remove [WVHT] 99.00 values

In [253]:
df23_clean = df23_clean.drop(df23_clean[df23_clean['WVHT'] == '99.00'].index)#removing any 99.00 values in WVHT
df23_clean.tail()

,WVHT
Date,
2023-12-31 19:40:00,2.76
2023-12-31 20:40:00,2.65
2023-12-31 21:40:00,2.5
2023-12-31 22:40:00,2.4
2023-12-31 23:40:00,2.87


In [254]:
df22_clean = df22_clean.drop(df22_clean[df22_clean['WVHT'] == '99.00'].index)#removing any 99.00 values in WVHT
df22_clean.tail()

,WVHT
Date,
2022-12-31 19:40:00,2.72
2022-12-31 20:40:00,2.46
2022-12-31 21:40:00,2.54
2022-12-31 22:40:00,2.5
2022-12-31 23:40:00,2.48


In [255]:
df21_clean = df21_clean.drop(df21_clean[df21_clean['WVHT'] == '99.00'].index)#removing any 99.00 values in WVHT
df21_clean.tail()

,WVHT
Date,
2021-12-31 19:40:00,3.02
2021-12-31 20:40:00,2.96
2021-12-31 21:40:00,2.92
2021-12-31 22:40:00,2.89
2021-12-31 23:40:00,2.93


In [256]:
df20_clean = df20_clean.drop(df20_clean[df20_clean['WVHT'] == '99.00'].index)#removing any 99.00 values in WVHT
df20_clean.tail()

,WVHT
Date,
2020-12-31 19:40:00,3.39
2020-12-31 20:40:00,3.18
2020-12-31 21:40:00,3.03
2020-12-31 22:40:00,3.19
2020-12-31 23:40:00,3.03


In [257]:
df19_clean = df19_clean.drop(df19_clean[df19_clean['WVHT'] == '99.00'].index)#removing any 99.00 values in WVHT
df19_clean.tail()

,Date,WVHT
8670,2019-12-31 18:50,4.40
8671,2019-12-31 19:50,5.43
8672,2019-12-31 20:50,4.54
8673,2019-12-31 21:50,4.66
8674,2019-12-31 22:50,5.25


#Merge all dataframes

In [269]:
df_2019_2023all = pd.concat([df23_clean, df22_clean, df20_clean, df20_clean, df19_clean], ignore_index=True, sort=False)
df_2019_2023all.head()


,WVHT,Date
0,2.61,NaN
1,2.59,NaN
2,2.69,NaN
3,2.55,NaN
4,2.28,NaN


In [270]:
import pandas as pd

def wrangle(df):
    # Skipping the first row
    df = df.iloc[1:]

    # Convert columns to string data type
    df = df.astype({'#YY': str, 'MM': str, 'DD': str, 'hh': str, 'mm': str})

    # Combine all the datetime info into a Date column
    df['Date'] = df['#YY'] + '-' + df['MM'] + '-' + df['DD'] + ' ' + df['hh'] + ':' + df['mm']

    # Display to check if the Date column is created correctly
    print(df[['#YY', 'MM', 'DD', 'hh', 'mm', 'Date']].head())

    # Deleting the old date and time columns
    df = df.drop(columns=['#YY', 'MM', 'DD', 'hh', 'mm'])

    # Convert 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # using errors='coerce' to handle any parsing errors

    # Display to check if the Date column is converted correctly
    print(df[['Date']].head())

    # Selecting columns
    df = df[['Date', 'WVHT']]

    # Setting the 'Date' column as the index
    df = df.set_index('Date')

    return df

# Apply the wrangle function to each DataFrame
df23_clean = wrangle(df23_clean)
df22_clean = wrangle(df22_clean)
df21_clean = wrangle(df21_clean)
df20_clean = wrangle(df20_clean)
df19_clean = wrangle(df19)

# Display the first few rows of each cleaned DataFrame to verify
print(df23_clean.head())
print(df22_clean.head())
print(df21_clean.head())
print(df20_clean.head())
print(df19_clean.head())


KeyError: "Only a column name can be used for the key in a dtype mappings argument. '#YY' not found in columns."